# Oracle AI Vector Search: 生成摘要

Oracle AI Vector Search 专为人工智能 (AI) 工作负载而设计，它允许您基于语义而不是关键词来查询数据。
Oracle AI Vector Search 的最大优势之一在于，可以将对非结构化数据的语义搜索与对业务数据的关系搜索在一个系统中相结合。
这不仅功能强大，而且效率显著更高，因为您无需添加专门的向量数据库，从而消除了多系统间数据碎片化的痛点。

此外，您的向量还可以受益于 Oracle 数据库所有最强大的功能，例如：

 * [分区支持](https://www.oracle.com/database/technologies/partitioning.html)
 * [Real Application Clusters 可伸缩性](https://www.oracle.com/database/real-application-clusters/)
 * [Exadata 智能扫描](https://www.oracle.com/database/technologies/exadata/software/smartscan/)
 * [跨地理分布式数据库的分片处理](https://www.oracle.com/database/distributed-database/)
 * [事务](https://docs.oracle.com/en/database/oracle/oracle-database/23/cncpt/transactions.html)
 * [并行 SQL](https://docs.oracle.com/en/database/oracle/oracle-database/21/vldbg/parallel-exec-intro.html#GUID-D28717E4-0F77-44F5-BB4E-234C31D4E4BA)
 * [灾难恢复](https://www.oracle.com/database/data-guard/)
 * [安全性](https://www.oracle.com/security/database-security/)
 * [Oracle 机器学习](https://www.oracle.com/artificial-intelligence/database-machine-learning/)
 * [Oracle 图数据库](https://www.oracle.com/database/integrated-graph-database/)
 * [Oracle Spatial and Graph](https://www.oracle.com/database/spatial/)
 * [Oracle 区块链](https://docs.oracle.com/en/database/oracle/oracle-database/23/arpls/dbms_blockchain_table.html#GUID-B469E277-978E-4378-A8C1-26D3FF96C9A6)
 * [JSON](https://docs.oracle.com/en/database/oracle/oracle-database/23/adjsn/json-in-oracle-database.html)

本指南演示了如何使用 Oracle AI Vector Search 中的摘要功能，通过 OracleSummary 为您的文档生成摘要。

如果你刚开始使用 Oracle Database，可以考虑探索 [免费的 Oracle 23 AI](https://www.oracle.com/database/free/#resources)，它能很好地帮助你入门数据库环境的搭建。在使用数据库时，通常建议避免默认使用 system 用户；相反，你可以创建自己的用户以增强安全性和自定义选项。关于用户创建的详细步骤，请参阅我们的 [端到端指南](https://github.com/langchain-ai/langchain/blob/master/cookbook/oracleai_demo.ipynb)，该指南也展示了如何在 Oracle 中设置用户。此外，理解用户权限对于有效管理数据库安全至关重要。你可以在官方的 [Oracle 指南](https://docs.oracle.com/en/database/oracle/oracle-database/19/admqs/administering-user-accounts-and-security.html#GUID-36B21D72-1BBB-46C9-A0C9-F0D2A8591B8D) 中了解更多关于管理用户账户和安全的信息。

### 前提条件

请安装 Oracle Python 客户端驱动程序，以便将 Langchain 与 Oracle AI Vector Search 结合使用。

In [ ]:
# pip install oracledb

### 连接到 Oracle 数据库
以下示例代码将展示如何连接到 Oracle 数据库。默认情况下，python-oracledb 在“Thin”模式下运行，该模式直接连接到 Oracle 数据库。此模式不需要 Oracle Client 库。然而，当 python-oracledb 使用这些库时会提供额外的功能。“Thick”模式指的是 python-oracledb 使用 Oracle Client 库时。两种模式都支持 Python Database API v2.0 规范的全面功能。请参阅以下[指南](https://python-oracledb.readthedocs.io/en/latest/user_guide/appendix_a.html#featuresummary)，其中介绍了每种模式支持的功能。如果您无法使用 Thin 模式，可以考虑切换到 Thick 模式。

In [ ]:
import sys

import oracledb

# please update with your username, password, hostname and service_name
username = "<username>"
password = "<password>"
dsn = "<hostname>/<service_name>"

try:
    conn = oracledb.connect(user=username, password=password, dsn=dsn)
    print("Connection successful!")
except Exception as e:
    print("Connection failed!")
    sys.exit(1)

### 生成摘要

Oracle AI Vector Search Langchain 库提供了一套专为文档摘要设计的 API。它支持多种摘要提供商，如 Database、OCIGENAI、HuggingFace 等，允许用户选择最符合其需求的提供商。要使用这些功能，用户必须按照指定配置摘要参数。有关这些参数的详细信息，请参阅 [Oracle AI Vector Search 指南](https://docs.oracle.com/en/database/oracle/oracle-database/23/arpls/dbms_vector_chain1.html#GUID-EC9DDB58-6A15-4B36-BA66-ECBA20D2CE57)。

***注意：*** 用户可能需要设置代理才能使用 Oracle 内部和默认提供商“database”之外的第三方摘要生成提供商。如果您没有代理，请在实例化 OracleSummary 时删除`proxy`参数。

In [ ]:
# proxy to be used when we instantiate summary and embedder object
proxy = "<proxy>"

以下示例代码将展示如何生成摘要：

In [ ]:
from langchain_community.utilities.oracleai import OracleSummary
from langchain_core.documents import Document

"""
# using 'ocigenai' provider
summary_params = {
    "provider": "ocigenai",
    "credential_name": "OCI_CRED",
    "url": "https://inference.generativeai.us-chicago-1.oci.oraclecloud.com/20231130/actions/summarizeText",
    "model": "cohere.command",
}

# using 'huggingface' provider
summary_params = {
    "provider": "huggingface",
    "credential_name": "HF_CRED",
    "url": "https://api-inference.huggingface.co/models/",
    "model": "facebook/bart-large-cnn",
    "wait_for_model": "true"
}
"""

# using 'database' provider
summary_params = {
    "provider": "database",
    "glevel": "S",
    "numParagraphs": 1,
    "language": "english",
}

# get the summary instance
# Remove proxy if not required
summ = OracleSummary(conn=conn, params=summary_params, proxy=proxy)
summary = summ.get_summary(
    "In the heart of the forest, "
    + "a lone fox ventured out at dusk, seeking a lost treasure. "
    + "With each step, memories flooded back, guiding its path. "
    + "As the moon rose high, illuminating the night, the fox unearthed "
    + "not gold, but a forgotten friendship, worth more than any riches."
)

print(f"Summary generated by OracleSummary: {summary}")

### 端到端演示

请参考我们完整的演示指南 [Oracle AI Vector Search 端到端演示指南](https://github.com/langchain-ai/langchain/tree/master/cookbook/oracleai_demo.ipynb)，借助 Oracle AI Vector Search 构建端到端的 RAG（检索增强生成）流程。